In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from fbprophet import Prophet

In [2]:
total_data=pd.read_csv("../data/total_data.csv",index_col=0)
total_data_demographic=pd.read_csv("../data/total_data_demographic.csv",index_col=0)

In [3]:
total_data[['customer_id','PRODUCT_ID','DAY']].groupby(['customer_id','PRODUCT_ID']).count().sort_values(by='DAY')

DAY
customer_id PRODUCT_ID     
1255        874736        1
1418        897125        1
            900802        1
            903325        1
            909714        1
...                     ...
900         890536      202
1998        1053690     202
1023        6533889     236
1510        894236      240
2337        1120741     240

[252243 rows x 1 columns]

In [4]:
a=total_data[['customer_id','DAY','PRODUCT_ID','WEEK_NO']]
a.customer_id=a.customer_id.transform(lambda x: str(x))

/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [5]:
encoded = pd.get_dummies(a.customer_id,prefix='bought_by_')
b=pd.concat([a,encoded],axis=1)

In [6]:
# b['last_purchase_day']=b.groupby(['customer_id','PRODUCT_ID'])['DAY'].shift()

In [7]:
b.drop('customer_id',inplace=True,axis=1)

In [8]:
b=b.groupby(['DAY','PRODUCT_ID','WEEK_NO'],as_index=False).sum()

In [9]:
product_data=pd.read_csv("../dataset/product.csv")

In [10]:
total_data=pd.merge(product_data[['PRODUCT_ID','DEPARTMENT','BRAND','COMMODITY_DESC','SUB_COMMODITY_DESC']],b,on='PRODUCT_ID')

In [11]:
total_data

,PRODUCT_ID,DEPARTMENT,BRAND,COMMODITY_DESC,SUB_COMMODITY_DESC,DAY,WEEK_NO,bought_by__1,bought_by__10,bought_by__100,...,bought_by__990,bought_by__991,bought_by__992,bought_by__993,bought_by__994,bought_by__995,bought_by__996,bought_by__997,bought_by__998,bought_by__999
0,819765,GROCERY,Private,CHEESE,NATURAL CHEESE EXACT WT CHUNKS,11,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,819765,GROCERY,Private,CHEESE,NATURAL CHEESE EXACT WT CHUNKS,21,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,819765,GROCERY,Private,CHEESE,NATURAL CHEESE EXACT WT CHUNKS,24,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,819765,GROCERY,Private,CHEESE,NATURAL CHEESE EXACT WT CHUNKS,26,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,819765,GROCERY,Private,CHEESE,NATURAL CHEESE EXACT WT CHUNKS,36,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263230,15926886,GROCERY,National,BAG SNACKS,POTATO CHIPS,707,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263231,15926886,GROCERY,National,BAG SNACKS,POTATO CHIPS,708,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263232,15926886,GROCERY,National,BAG SNACKS,POTATO CHIPS,709,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263233,15926886,GROCERY,National,BAG SNACKS,POTATO CHIPS,710,102,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# for id in set([1004906]):
#     column_name='buys_'+str(id)
#     is_product=pd.DataFrame(total_data.PRODUCT_ID==id).rename(columns={'PRODUCT_ID':column_name})
#     total_data=pd.concat([total_data[['customer_id', 'DAY', 'WEEK_NO', 'days_since_last_purchase']], is_product], axis=1)

In [13]:
# total_data['ds']=total_data.DAY
# total_data.drop('DAY',axis=1,inplace=True)

In [14]:
# m=Prophet()
# m.fit(total_data)

In [15]:
# future_df=m.make_future_dataframe(periods=31)

In [16]:
# forecast=m.predict(future_df)

In [23]:
def train_test_split(X,y,test_size=0.25,val_size=.2):
    X_test_Split=int((X.shape[0])*(test_size+val_size))
    X_val_Split=int((X.shape[0])*val_size)
    y_test_Split=int((y.shape[0])*(test_size+val_size))
    y_val_Split=int((y.shape[0])*val_size)
    print(X_test_Split,X_val_Split,y_test_Split,y_val_Split)
    X_Train=X[: -X_test_Split]
    X_Test=X[-X_test_Split:-X_val_Split]
    X_Val=X[-X_val_Split:]
    y_Train=y[: -y_test_Split]
    y_Test=y[-y_test_Split:-y_val_Split]
    y_Val=y[-y_val_Split:]
    return [X_Train, X_Test,X_Val, y_Train, y_Test,y_Val]

In [24]:
#one hot encoding categorical data for modelling
encoded = pd.get_dummies(total_data[total_data.columns[total_data.dtypes==object]])
total_data_encoded = pd.concat([encoded, total_data[total_data.columns[total_data.dtypes != object]]],axis=1)

X = total_data_encoded.drop('bought_by__2337',axis=1)
y = total_data_encoded['bought_by__2337']

[X_Train, X_Test,X_Val, y_Train, y_Test,y_Val] = train_test_split(X,y,test_size=0.2,val_size=.2)

105294 52647 105294 52647


In [19]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#XGBoost
xgb_mdl = XGBClassifier(nthread=-1).fit(X_Train.values,y_Train.values)
xgb_mdl

NameError: name 'X_Train' is not defined

In [20]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, average_precision_score

y_pred_train = xgb_mdl.predict(X_Train.values)
y_pred = xgb_mdl.predict(X_Test.values)

print("Train Data Classification Report:\n")
print(classification_report(y_Train,y_pred_train))

print("Test Data Classification Report:\n")
print(classification_report(y_Test,y_pred))

cm = pd.DataFrame(confusion_matrix(y_Test, y_pred))#, index = list(set(y)), columns = list(set(y)))


NameError: name 'xgb_mdl' is not defined

In [21]:
cm

NameError: name 'cm' is not defined

In [22]:
set(y_Test)

NameError: name 'y_Test' is not defined